# Lesson 4: Evaluate the Tuned Model

#### Project environment setup

- Install Tensorboard (if running locally)
```Python
!pip install tensorboard
```

### Explore results with Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
port = %env PORT1
%tensorboard --logdir reward-logs --port $port --bind_all 

In [ ]:
# Look at what this directory has
%ls reward-logs

In [ ]:
port = %env PORT2
%tensorboard --logdir reinforcer-logs --port $port --bind_all

In [ ]:
port = %env PORT3
%tensorboard --logdir reinforcer-fulldata-logs --port $port --bind_all

- The dictionary of 'parameter_values' defined in the previous lesson

In [ ]:
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 1410,
        "reinforcement_learning_train_steps": 320,
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 1.0,
        "kl_coeff": 0.1,
        "instruction":\
    "Summarize in less than 50 words"}

**Note:** Here, we are using "text_small" for our datasets for learning purposes. However for the results that we're evaluating in this lesson, the team used the full dataset with the following hyperparameters:

```Python
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 10000,
        "reinforcement_learning_train_steps": 10000, 
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 0.2,
        "kl_coeff": 0.1,
        "instruction":\
    "Summarize in less than 50 words"}
```

### Evaluate using the tuned and untuned model

In [ ]:
import json

In [ ]:
eval_tuned_path = 'eval_results_tuned.jsonl'

In [ ]:
eval_data_tuned = []

In [ ]:
with open(eval_tuned_path) as f:
    for line in f:
        eval_data_tuned.append(json.loads(line))

In [ ]:
# Import for printing purposes
from utils import print_d

In [ ]:
# Look at the result produced by the tuned model
print_d(eval_data_tuned[0])

In [ ]:
eval_untuned_path = 'eval_results_untuned.jsonl'

In [ ]:
eval_data_untuned = []

In [ ]:
with open(eval_untuned_path) as f:
    for line in f:
        eval_data_untuned.append(json.loads(line))

In [ ]:
# Look at the result produced by the untuned model
print_d(eval_data_untuned[0])

### Explore the results side by side in a dataframe

In [ ]:
# Extract all the prompts
prompts = [sample['inputs']['inputs_pretokenized']
           for sample in eval_data_tuned]

In [ ]:
# Completions from the untuned model
untuned_completions = [sample['prediction']
                       for sample in eval_data_untuned]

In [ ]:
# Completions from the tuned model
tuned_completions = [sample['prediction']
                     for sample in eval_data_tuned]

- Now putting all together in one big dataframe

In [ ]:
import pandas as pd

In [ ]:
results = pd.DataFrame(
    data={'prompt': prompts,
          'base_model':untuned_completions,
          'tuned_model': tuned_completions})

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Print the results
results